# Instructions
Run the notebook except the last cell to load all the packages, models and functions. Then, modify the `user_sentence` string variable in the last cell of the notebook; set it to be the sentence you would like the model to predict a relation for. Your sentence must contain tags to indicate the entities of the relation you would like the model to predict. For example your sentence may be: `'The <e1>bottle</e1> is filled with <e2>water</e2>.'`. The `'<e1>'` and `'</e1>'` tags indicate the first entity, `'bottle'`, and the `'<e2>'` and `'</e2>'` tags indicate the second entity, `'water'`.

# Install Packages

In [ ]:
!pip install numpy torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

# Import Packages and Load Models

In [ ]:
import json
import pickle

import numpy as np
import torch
from transformers import BertTokenizer, BertModel

# path = '/'
from google.colab import drive
path = '/content/drive/MyDrive/Text_Mining/BERT-SVM/'
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open(path + 'Models/svm_best_model.pickle', 'rb') as fp:
    loaded_svm_model = pickle.load(fp)
bert_model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)
bert_model.eval()
device

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

device(type='cpu')

# List of Relations Mapped by Numeric Labels

In [ ]:
label_names = {
    0: 'Cause-Effect(e1,e2)',
    1: 'Cause-Effect(e2,e1)',
    2: 'Component-Whole(e1,e2)',
    3: 'Component-Whole(e2,e1)',
    4: 'Content-Container(e1,e2)',
    5: 'Content-Container(e2,e1)',
    6: 'Entity-Destination(e1,e2)',
    7: 'Entity-Destination(e2,e1)',
    8: 'Entity-Origin(e1,e2)',
    9: 'Entity-Origin(e2,e1)',
    10: 'Instrument-Agency(e1,e2)',
    11: 'Instrument-Agency(e2,e1)',
    12: 'Member-Collection(e1,e2)',
    13: 'Member-Collection(e2,e1)',
    14: 'Message-Topic(e1,e2)',
    15: 'Message-Topic(e2,e1)',
    16: 'Product-Producer(e1,e2)',
    17: 'Product-Producer(e2,e1)',
    18: 'Other',
}

# Try out the model

In [ ]:
def predict_relation(sentence, tokenizer, bert_model, svm_model, label_names, device):

    # Tokenize the input sentence
    encoded = tokenizer(sentence, padding=True, truncation=True, return_tensors="pt")

    # Move tensors to the correct device (GPU or CPU)
    input_ids = encoded["input_ids"].to(device)
    attention_mask = encoded["attention_mask"].to(device)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = bert_model(input_ids, attention_mask=attention_mask)

    sentence_embedding = outputs.last_hidden_state.mean(dim=1)
    X_new = sentence_embedding.cpu().numpy()

    # Predict label using the SVM model
    predicted_label_idx = svm_model.predict(X_new)[0]

    # Convert numeric label back to relation name
    predicted_relation = label_names[predicted_label_idx]

    return predicted_relation


In [ ]:
user_sentence = 'The <e1>bottle</e1> is filled with <e2>water</e2>.'

print(predict_relation(user_sentence, tokenizer, bert_model, loaded_svm_model, label_names, device))

Content-Container(e2,e1)
